In [13]:
import numpy as np
import re
import _pickle as pickle
from bs4 import BeautifulSoup
import pymorphy2
import os
from IPython.display import clear_output

In [14]:
def make_stop_set():
    s = set()
    with open('./stop/stopwords.txt', 'r') as f:
        line = f.readline()[:-1]
        while line: 
            s.add(line)
            line = f.readline()[:-1]
    return s

In [15]:
def url_doc():
    with open ('urls.numerate.txt') as f:
        line = True
        u_d = {}
        while line:
            line = f.readline()
            if line:
                id_url = line.split()
                u_d[id_url[1]] = id_url[0]
    return u_d

In [16]:
"""
Получаем словарь URL <-> DOCID

"""
URL_DOC = url_doc()

In [17]:
morph=pymorphy2.MorphAnalyzer()

In [24]:
"""
Сохранение/открытие словарей


"""
def save_dict(dict_, docid):
    with open('./docid_dicts/'+docid+'.pkl', 'wb') as out:
        pickle.dump(dict_, out, 2)
        
def open_dict(docid):
    with open('./docid_dicts/'+docid+'.pkl', 'rb') as inp:
        d = pickle.load(inp)
    return d

def open_public(docid):
    with open('./dicts_for_spell/'+docid+'.pkl', 'rb') as inp:
        d = pickle.load(inp)
    return d

def open_bigramm(docid):
    with open('./bigramms_for_spell/'+docid+'.pkl', 'rb') as inp:
        d = pickle.load(inp)
    return d

In [25]:
"""
Функция проходит по всем папкам с документами и открывает каждый

"""

def open_files():
    folders = ['./content/content/20170702/', './content/content/20170704/', './content/content/20170707/',\
               './content/content/20170708/', './content/content/20170710/', './content/content/20170711/',\
               './content/content/20170717/', './content/content/20170726/']
    i = 0
    for cur_folder in folders:
        for file in os.listdir(cur_folder):
            if i % 100 == 0:
                print('FILES LOADED: ', i)
            parser(cur_folder+file)
            i+=1            

In [28]:
"""
Здесь происходит парсинг документа, лежащего по заданному пути

Полцчаем URL из первой строки, сопоставляем его с DOCID при помощи словаря URL <-> DOCID

При помощи bs4 находим содержимое тегов, чистим при помощи regexp от мусора

Производим лемматизацию слов документа, сохраняем в словарь вида: TAG <-> WORD <-> FREQ

Что важно - мы сохраняем словарь в виде DOCID.pkl, благодаря этому легко сможем
получить доступ к интересующему нас словарю по DOCID

"""

def parser(way):
    
    dict_ = {}
    
    with open(way, encoding='utf-8', errors='ignore') as file:
        url = file.readline()[:-1]
        
        raw_html = file.read()
        soup = BeautifulSoup(raw_html, 'html')
        
        for obj in soup.find_all(['body', 'title', 'h1', 'h2', 'a', 'meta']):
            
            tag = obj.name
            
            text = obj.text
            text = re.sub('\W+', ' ', text.lower())
            words_list = text.split()
            
            if tag == 'meta':   
                if obj.get('content') != None:
                    new_text = obj.get('content')
                    new_text = re.sub('\W+', ' ', new_text.lower())
                    new_text = re.sub('[a-z]', ' ', new_text)
                    new_words = new_text.split()
                    words_list += new_words
                    

                    
            for string in words_list:
                
                lemmatized = morph.parse(string)[0].normal_form
                
                if tag in dict_:
                    if lemmatized in dict_[tag]:
                        dict_[tag][lemmatized] += 1
                    else:
                        dict_[tag][lemmatized] = 1
                else:
                    dict_[tag] = {lemmatized : 1}
#     return dict_
    save_dict(dict_, URL_DOC[url])

In [31]:
"""
Словарь: QuerryNumber <-> [DOCID_1, ..., DOCID_N]

"""
def qnum_docs():
    dict_ = {}
    with open('sample.technosphere.ir1.textrelevance.submission.txt') as f:
        line = f.readline()
        while line:
            line = f.readline()
            if line:
                line = line[:-1].split(',')
                if line[0] in dict_:
                    dict_[line[0]] += [line[1]]
                else:
                    dict_[line[0]] = [line[1]]
    return dict_

In [39]:
"""
Словарь: DOCID <-> TAG <-> LENGTH

"""

def get_doc_tag_len():
    doc_tag_len = {}
    i=0
    for docid in range(1, 38115):
        if not i % 100:
            clear_output(wait=True)
            print('Files handled: {}; Files left: {}'.format(i,38114-i))
        cur_doc = open_dict(str(docid))
        i+=1
        for tag in cur_doc:

            for word in cur_doc[tag]:

                if str(docid) in doc_tag_len:
                    if tag in doc_tag_len[str(docid)]:
                        doc_tag_len[str(docid)][tag] += cur_doc[tag][word]
                    else:
                        doc_tag_len[str(docid)][tag] = cur_doc[tag][word]
                else:
                    doc_tag_len[str(docid)] = {tag : cur_doc[tag][word]}
                        
    return doc_tag_len       

In [40]:
"""
Словарь: QuerryNumber <-> TAG <-> AverageLength

"""

def qnum_len():
    avg = {}
    
    for qnum in QNUM_D:
        
        counter = {'body':0, 'title':0, 'h1':0, 'h2':0, 'a':0, 'meta':0}
        
        for docid in QNUM_D[qnum]:
            
            if docid in DOC_TAG_L:
                for tag in DOC_TAG_L[docid]:
                    counter[tag] += DOC_TAG_L[docid][tag]

            
        for key in counter:
            counter[key] = counter[key] / len(QNUM_D[qnum])
            
        avg[qnum] = counter
            
    return avg

In [41]:
"""
Словарь: QuerryNumber <-> [lemmatized_1, ..., lemmatized_N]
То есть список лемматизированных слов запроса

"""
def requests():
    dict_ = {}
    with open('queries.numerate.txt', encoding='utf8') as f:
        line = f.readline().lower()
        while line:
            list_ = line.split()
            lemmatized = [morph.parse(word)[0].normal_form for word in list_[1:]]
            dict_[list_[0]] = lemmatized
    
            line = f.readline().lower()
            
    return dict_

In [42]:
%%time
QNUM_Q = requests()

CPU times: user 775 ms, sys: 0 ns, total: 775 ms
Wall time: 773 ms


In [43]:
%%time
QNUM_D = qnum_docs()

CPU times: user 27.6 ms, sys: 18.8 ms, total: 46.4 ms
Wall time: 41.4 ms


In [44]:
%%time
DOC_TAG_L = get_doc_tag_len()

Files handled: 38100; Files left: 14
CPU times: user 2min 46s, sys: 6.28 s, total: 2min 52s
Wall time: 3min 52s


In [45]:
%%time
Q_TAG_AVG = qnum_len()

CPU times: user 108 ms, sys: 9.97 ms, total: 118 ms
Wall time: 112 ms


In [46]:
"""
Вспомогательная функция для вывода полученного решения

"""
def print_submmission(final_dict):
    for qnum in final_dict:
        res = list(final_dict[qnum].items())
        res.sort(key=lambda i: i[1])
        res = res[-10:]
        for i in res[::-1]:
            print(str(qnum)+','+str(i[0])+'    '+str(np.around(i[1],4))+'\n')

In [47]:
"""
Здесь мы применяем BM25F для оценки релевантности документа

Будем применять не модель не к документу целиком, а к каждому тегу в документе

Далее для каждого тега мы можем прикинуть его значимость
(т.е. будем получать некую оценку от каждого тега, а далее учитывать ее с некоторым весом)


"""
def get_score():
    qnum_doc_tag_score = {}
    idf = {}
    score = {}
    additional_docs = []

    for qnum in QNUM_Q:
        docs = QNUM_D[qnum]
        
        if int(qnum) < 200:
            additional_docs = []
            for i in [int(qnum)+1, int(qnum)+2, int(qnum)+3, int(qnum)+4]: # int(qnum)+5]:
                additional_docs += QNUM_D[str(i)]
                
        elif int(qnum) >= 200:
            additional_docs = []
            for i in [int(qnum)-1, int(qnum)-2, int(qnum)-3, int(qnum)-4]:#, int(qnum)-5]:
                additional_docs += QNUM_D[str(i)]
            
            
        # Временныйы словарь частот слов по тегам для очередного запроса
        tmp = {}
        for docid in docs+additional_docs:
            tmp[docid] = open_dict(docid)
        
        for word in QNUM_Q[qnum]:
            
            n_q_i = {'body':0, 'title':0, 'h1':0, 'h2':0, 'a':0, 'meta':0}
            N = len(docs+additional_docs)
            
            # Подсчет документов-тегов, в которых есть терм
            for docid in tmp:
                for tag in tmp[docid]:
                    if word in tmp[docid][tag]:
                        n_q_i[tag] += 1

            # Подсчет IDF для очередного терма запроса
            for tag in ['body', 'title', 'h1', 'h2', 'a', 'meta']:
                tmp_idf = np.log((N-n_q_i[tag]+0.5) / (n_q_i[tag]+0.5)) #tag -> body?
                if tmp_idf > 0:
                    idf[tag] = tmp_idf
                else:
                    idf[tag] = 0
                    
            #Подсчёт финального скора для каждого документа
            for docid in docs:
                for tag in tmp[docid]:
                    if word in tmp[docid][tag]:
                        
                        b = 0.75
                        
                        if tag in ['title']:
                            w = 2
                        
                        elif tag in ['h1', 'meta']:
                            w = 1.5
                            
                        elif tag in ['body', 'h2']:
                            w = 0.5
                            
                        elif tag in ['a']:
                            w = 0.01
                            
                        else:
                            w = 0
                            
                        TF = w*tmp[docid][tag][word]
                        
                        score[tag] = idf[tag]*((TF*(2+1))/\
                                    (TF+2*(1-b+b*(DOC_TAG_L[docid][tag]/Q_TAG_AVG[qnum][tag]))))
                        
                        if qnum in qnum_doc_tag_score:
                            if docid in qnum_doc_tag_score[qnum]:
                                if tag in qnum_doc_tag_score[qnum][docid]:
                                    qnum_doc_tag_score[qnum][docid][tag] += score[tag]
                                else:
                                    qnum_doc_tag_score[qnum][docid][tag]  = score[tag]
                            else:
                                qnum_doc_tag_score[qnum][docid] = {tag : score[tag]}
                        else:
                            qnum_doc_tag_score[qnum] = {docid : {tag: score[tag]}}
        clear_output(wait=True)
        print('{} <-- requests ready'.format(qnum))
    
    return qnum_doc_tag_score

In [598]:
%%time
final_dict = get_score()

399 <-- requests ready
CPU times: user 2min 31s, sys: 6.28 s, total: 2min 37s
Wall time: 2min 36s


In [604]:
FINAL = result(final_dict)

In [605]:
def make_submission(final_dict):
    with open('./submissions/'+'tag_a_01.txt', 'w') as f:
        f.write('QueryId,DocumentId\n')
        for qnum in final_dict:
            res = list(final_dict[qnum].items())
            res.sort(key=lambda i: i[1])
            res = res[-10:]
            for i in res[::-1]:
                print(str(qnum)+','+str(i[0])+'\n')
                str(qnum)+','+str(i[0])+'\n'
                f.write(str(qnum)+','+str(i[0])+'\n')

In [606]:
def result(final_dict):
    qnum_doc_score = {}

    for qnum in final_dict:
        for doc in final_dict[qnum]:
            for tag in final_dict[qnum][doc]:
                
                w = 1
                    
                if qnum in qnum_doc_score:
                    if doc in qnum_doc_score[qnum]:
                        qnum_doc_score[qnum][doc] += w*final_dict[qnum][doc][tag]
                    else:
                        qnum_doc_score[qnum][doc] = w*final_dict[qnum][doc][tag]
                else:
                    qnum_doc_score[qnum] = {doc : w*final_dict[qnum][doc][tag]}

    return qnum_doc_score            

In [56]:
# make_submission(FINAL)

In [48]:
"""
Далее идут функции, реализующие SpellChecker.
Составляем словарь из всех слов во всех документах
Составляем словари биграмм по сотням, т.к. память не резиновая :)

"""

from itertools import islice
import pkg_resources
from symspellpy import SymSpell, Verbosity

In [49]:
def get_public_freq_dict():
    public_d = {}
    i=0
    for i in range(1, 38115):
        if not i%100:
            print('{} <--- loaded'.format(i))
        docid = str(i)
        curr_dict = open_public(docid)
        for tag in curr_dict:
            for word in curr_dict[tag]:
                if word and len(word)<18:
                    if word[0]>='a' and word[0]<='z' or word[0]>='а' and word[0]<='я' or word[0]>='0' and word[0]<='9':
                        if word in public_d:
                            public_d[word] += curr_dict[tag][word]
                        else:
                            public_d[word] = curr_dict[tag][word]
                    
    with open('./freq_dicts/public.txt', 'w') as f:
        for word in public_d:
            if public_d[word] > 3:
                f.write(word+' '+str(public_d[word])+'\n')

In [30]:
def get_public_bigr_dict():
    public_d={}
    i=0
    for qnum in QNUM_D:

        for docid in QNUM_D[qnum]:
            docid = str(docid)
            curr_dict = open_bigramm(docid)
            for tag in curr_dict:
                for bigramm in curr_dict[tag]:
                    tmp = bigramm.split()
                    if len(tmp[0])<18 and len(tmp[1])<18:
                        if tmp[0][0]>='a' and tmp[0][0]<='z' or tmp[0][0]>='а' and tmp[0][0]<='я':
                            if tmp[1][0]>='a' and tmp[1][0]<='z' or tmp[1][0]>='а' and tmp[1][0]<='я':
                                if bigramm in public_d:
                                    public_d[bigramm] += curr_dict[tag][bigramm]
                                else:
                                    public_d[bigramm] = curr_dict[tag][bigramm]
        
        clear_output(wait=True)
        print('Querry {} is ready'.format(qnum))
        if int(qnum) in {100, 200, 300, 399}:
            with open('./freq_dicts/bigramms_dict'+qnum+'.txt', 'w') as f:
                for bigramm in public_d:
                    if public_d[bigramm] > 1:
                        f.write(bigramm+' '+str(public_d[bigramm])+'\n')
            public_d = {}

In [53]:
"""
Здесь реализация split, join, transletiration, orphography

"""

def correct(s):
    final_s = ''
    s = sym_spell.word_segmentation(s) 
    s = str(s[0])
    l = s.split()
    tmp_s = ''
    f = True
    ended = True
    for i in range(len(l[:-1])):
        if f:
            suggestions = sym_spell.lookup(l[i]+l[i+1], Verbosity.ALL,
                                max_edit_distance=2, include_unknown=True)
            if suggestions:
                if int(str((suggestions[0])).split(', ')[2]) > 200 and int(str((suggestions[0])).split(', ')[1]) == 0:
                    tmp_s += str(suggestions[0]).split(', ')[0] + ' '
                    f = False
                    if i+1 == len(l)-1:
                        ended = False
                else:
                    tmp_s += l[i]+' '
        else:
            f = True
    if ended:
        tmp_s += l[-1]
    
    s = tmp_s
        
    for i in s.split():
        suggestions = sym_spell.lookup(i, Verbosity.ALL,
                                max_edit_distance=2, include_unknown=True)
        res = []
        for suggestion in suggestions:
            res += [str(suggestion).split(', ')]
        res.sort(key=lambda i: 0.3*np.log(int(i[2])) - int(i[1]))
        final_s += res[::-1][0][0] + ' '
        
    return final_s[:-1]

In [54]:
def make_transletiration(s):
    eng_rus = {'q':'й', 'w':'ц', 'e':'у', 'r':'к', 't':'е', 'y':'н'\
           , 'u':'г', 'i':'ш', 'o':'щ', 'p':'з','[':'х',']':'ъ','a':'ф',\
           's':'ы','d':'в','f':'а','g':'п','h':'р','j':'о','k':'л'\
           ,'l':'д',';':'ж','\'':'э','z':'я','x':'ч','c':'с','v':'м','b':'и','n':'т','m':'ь',',':'б','.':'ю'}
    res = ''
    l = s.split()
    for i in l:
        for j in i:
            res += eng_rus[j]
        res += ' '
    return res[:-1]

In [55]:
# SPELLCHECKER -> {TRANSLITERATION, SPLIT, JOIN} -> ORPHOGRAPHY
# sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
# sym_spell.load_dictionary('./freq_dicts/public.txt', term_index=0, count_index=1)

def get_newQ():
    newQ={}

    with open('queries.numerate.txt','r') as f:
        line = f.readline()
        while line:
            l = line.split()
            clear_output(wait=True)
            print(l[0])
            s = ''
            for i in l[1:]:
                s += i + ' '
            s = s[:-1]
            
            if l[1][0]>='a' and l[1][0] <= 'z':
                s = make_transletiration(s)
                
            elif l[2:]:
                if l[2][0]>='a' and l[2][0]<='z' and l[-1][0]>='a' and l[-1][0]<='z' :
                    s = ''
                    for i in l[2:]:
                        s += i + ' '
                    s = s[:-1]
                    s = l[1] + ' ' + make_transletiration(s)
            else:
                pass
            
            s = correct(s)
            
            if l[0] in {'1','100','200','300'}: 
                sym_spell.load_bigram_dictionary('./freq_dicts/bigramms_dict'+l[0]+'.txt', term_index=0, count_index=2)

            suggestions = sym_spell.lookup_compound(s, max_edit_distance=2)
            res = str(suggestions[0]).split(', ')
            newQ[l[0]] = res[0]
            line = f.readline()
            
    with open('queries.numerate.new.txt','w') as f:
        for q in newQ:
            f.write(q+'\t'+newQ[q]+'\n')

**Подводя итог:** 

- Удалось добиться достаточно хорошего результата, используя классическую модель BM25F
- Подбирать веса без train - проблематично, но +- правильно подобранные веса очень даже неплохо прибавляют 
- Больше всего скор вырос после добавления лемматизации. Пробовал стемминг, но вышло хуже (мб сам где ошибся)
- Очень неплохо подбросил spellchecker
- Ещё была идея пройтись по документам антиспамом и понижать, если антиспам сработал, но пока что не успел